In [ ]:
# Load necessary modules
import os
import glob
import numpy as np
import pandas as pd
import MDAnalysis as mda
import matplotlib.pyplot as plt
import MDAnalysis.lib.util as util # To convert from 3 letter to 1 letter aa code
import matplotlib.patheffects as pe # Better viewing in plotting

from MDAnalysis.analysis import distances
from MDAnalysis.analysis import contacts
import seaborn as sns
import matplotlib.colors as mcolors

# For Clustering analysis
from MDAnalysis.analysis import encore
from MDAnalysis.analysis.encore.clustering import ClusteringMethod as clm

In [ ]:
# Detect all folders and provide correct names
folders = glob.glob('?.*')
folders.sort(reverse=False)
filenames = ['0_Control','1_NKA1_ab2','2_NKA2_ab2','3_NKAW1_2ab','4_NKAW2_2ab']
peptides = ['Control','NKA1_Aβ2','NKA2_Aβ2','NKAW1_Aβ2','NKAW2_Aβ2']
# Colors are:    
ticks_colors = ['magenta','green','blue','orange']
data_folder = './AnalysisData/'
plots_folder = './AnalysisPlots/'

In [ ]:
# Get correct files and folders for each peptide
pept_num = 1
folder = f'{folders[pept_num]}/amber/'
print(f'Analysis will be run for: {folder}')
parm = f'{folder}firstframe.pdb'
traj = f'{folder}last500ns.nc'
filename = f'{filenames[pept_num]}'
peptide = peptides[pept_num]

In [ ]:
# Load universe
u = mda.Universe(parm,traj)

## Clustering

In [ ]:
def clustering(peptide):
    """
    This function clusters a trajectory into a single cluster, and outputs the centroid into a CRD file
    """
    # Do the clustering
    clustering_method = clm.AffinityPropagationNative(preference=-50.0,damping=0.9,max_iter=200,convergence_iter=30,add_noise=True)
    ces1, details1 = encore.ces([u],select='name CA',clustering_method=clustering_method)

    # Get centroid id
    cluster_collection = details1['clustering'][0]
    centroid = cluster_collection.get_centroids()

    # We just want to have a centroid
    if len(centroid) > 1:
        print(f'Please revise clustering of {peptide}')

    # Write PDB file with centroid
    u.trajectory[centroid[0]]
    u.select_atoms('protein').write(f'{data_folder}{peptide}_centroid.pdb')

## Heatmap

### Distances

In [ ]:
def add_colors_to_letters(ax,fontsize,do_xlabel=False,do_ylabel=False):
    """ This function is used to color the letters """
    if do_ylabel:
        for tick in ax.get_yticklabels():
            if tick.get_position()[1] < 15:
                tick.set_color(ticks_colors[0])
            elif tick.get_position()[1] < 22:
                tick.set_color(ticks_colors[1])
            elif tick.get_position()[1] < 30:
                tick.set_color(ticks_colors[2])
            else:
                tick.set_color(ticks_colors[3])
            tick.set_fontweight('bold')
            tick.set_fontsize(fontsize)
    if do_xlabel:
        for tick in ax.get_xticklabels():
            if tick.get_position()[0] < 15:
                tick.set_color(ticks_colors[0])
            elif tick.get_position()[0] < 22:
                tick.set_color(ticks_colors[1])
            elif tick.get_position()[0] < 30:
                tick.set_color(ticks_colors[2])
            else:
                tick.set_color(ticks_colors[3])
            tick.set_fontweight('bold')
            tick.set_fontsize(fontsize)

In [ ]:
def distances_protein_protein(u,chain1,chain2,filename=filename,data_folder=data_folder,peptide=peptide,plots_folder=plots_folder,fontsize=18,cmap_color='RdBu'):
    """ This function analyses the distance between two AB42 chains """
    # Calculate distances between chains
    n_frames = len(u.trajectory)
    ca1 = chain1.select_atoms('name CA')
    ca1_resn = ca1.resnames
    ca1_resn_one = [mda.lib.util.convert_aa_code(i) for i in ca1_resn]
    ca2 = chain2.select_atoms('name CA')
    ca2_resn = ca2.resnames
    ca2_resn_one = [mda.lib.util.convert_aa_code(i) for i in ca2_resn]
    distances = np.zeros((42, 42), dtype=float)
    for _ in u.trajectory:
        dist_matrix = mda.analysis.distances.distance_array(ca1.positions, ca2.positions)
        distances += dist_matrix

    # Calculate average distance over all frames
    distances /= n_frames

    # Save to df
    df_distances = pd.DataFrame(distances)
    df_distances.to_csv(f'{data_folder}1.Heatmaps/df_{filename}_ab1_ab2_distance.csv')

    # Generate distance heatmap with colorbar
    fig, ax = plt.subplots(figsize = (13,13))
    sns.heatmap(distances, cmap=cmap_color, ax=ax, cbar_kws={'label': 'Distance (Å)', "shrink": .82}, 
                square=True, xticklabels=ca1_resn_one, yticklabels=ca2_resn_one, 
                vmin=0, vmax=50, linewidths=0.01, linecolor='white', cbar=True)
    ax.invert_yaxis()
    # ax.set_title(f'Aβ1/Aβ2 distance per residue ({peptide})', fontdict={'fontsize':24, 'fontweight':'bold'})
    # ax.set_xlabel('Aβ2 Residue Index', fontdict={'fontsize':22, 'fontweight':'bold'})
    # ax.set_ylabel('Aβ1 Residue Index', fontdict={'fontsize':22, 'fontweight':'bold'})
    ax.figure.axes[-1].yaxis.label.set_size(18) #Change size of cbar label
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)  #Change size of cbar numbers
    #Add colors to letters
    add_colors_to_letters(ax,fontsize,do_xlabel=True,do_ylabel=True)

    plt.tight_layout()
    plt.savefig(f'{plots_folder}1.Heatmaps/{filename}_ab1_ab2_distance.svg')
    plt.close()

In [ ]:
def distances_peptide_abeta1(u,chain1,chain2,chain3='',filename=filename,data_folder=data_folder,peptide=peptide,plots_folder=plots_folder,fontsize=18,cmap_color='RdBu'):
    """ This function analyses the distance between the peptide and AB42 chain 1 """
    # Calculate distances between chains
    n_frames = len(u.trajectory)
    ca1 = chain1.select_atoms('name CA')
    ca2 = chain3.select_atoms('name CA')
    ca1_resn = ca1.resnames
    ca2_resn = ca2.resnames
    ca1_resn_one = [mda.lib.util.convert_aa_code(i) for i in ca1_resn]
    ca2_resn_one = [mda.lib.util.convert_aa_code(i) for i in ca2_resn]

    distances = np.zeros((len(ca1), len(ca2)), dtype=float)
    for ts in u.trajectory:
        dist_matrix = mda.analysis.distances.distance_array(ca1.positions, ca2.positions)
        distances += dist_matrix

    # Calculate average distance over all frames
    distances /= n_frames

    # Save to df
    df_distances = pd.DataFrame(distances)
    df_distances.to_csv(f'{data_folder}1.Heatmaps/df_{filename}_ab1_peptide_distance.csv')

    # Generate distance heatmap with colorbar
    fig, ax = plt.subplots(figsize=(13,13))
    sns.heatmap(distances, cmap=cmap_color, ax=ax, cbar_kws={'label': 'Distance (Å)', "shrink": .82}, 
                xticklabels=ca2_resn_one, yticklabels=ca1_resn_one, 
                vmin=0, vmax = 50, center=10, linewidths=0.01, linecolor='white', cbar=True)
    ax.invert_yaxis()
    # ax.set_title(f'Aβ1/NKAW distance per residue {peptide}', fontdict={'fontsize':24, 'fontweight':'bold'})
    # ax.set_xlabel('NKAW Residue Index', fontdict={'fontsize':22, 'fontweight':'bold'})
    # ax.set_ylabel('Aβ1 Residue Index', fontdict={'fontsize':22, 'fontweight':'bold'})
    ax.figure.axes[-1].yaxis.label.set_size(18) #Change size of cbar label
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)  #Change size of cbar numbers    
    add_colors_to_letters(ax,fontsize,do_ylabel=True)
    for tick in ax.get_xticklabels():
        tick.set_fontweight('bold')
        tick.set_fontsize(fontsize)
    
    plt.tight_layout()
    plt.savefig(f'{plots_folder}1.Heatmaps/{filename}_ab1_peptide_distance.svg')
    plt.close()

In [ ]:
def distances_peptide_abeta2(u,chain1,chain2,chain3='',filename=filename,data_folder=data_folder,peptide=peptide,plots_folder=plots_folder,fontsize=18,cmap_color='RdBu'):
    """ This function analyses the distance between the peptide and AB42 chain 2 """
    # Calculate distances between chains
    n_frames = len(u.trajectory)
    ca1 = chain2.select_atoms('name CA')
    ca2 = chain3.select_atoms('name CA')
    ca1_resn = ca1.resnames
    ca2_resn = ca2.resnames
    ca1_resn_one = [mda.lib.util.convert_aa_code(i) for i in ca1_resn]
    ca2_resn_one = [mda.lib.util.convert_aa_code(i) for i in ca2_resn]

    distances = np.zeros((len(ca1), len(ca2)), dtype=float)
    for ts in u.trajectory:
        dist_matrix = mda.analysis.distances.distance_array(ca1.positions, ca2.positions)
        distances += dist_matrix

    # Calculate average distance over all frames
    distances /= n_frames

    # Save to df
    df_distances = pd.DataFrame(distances)
    df_distances.to_csv(f'{data_folder}1.Heatmaps/df_{filename}_ab2_peptide_distance.csv')

    # Generate distance heatmap with colorbar
    fig, ax = plt.subplots(figsize=(13,13))
    sns.heatmap(distances, cmap=cmap_color, ax=ax, cbar_kws={'label': 'Distance (Å)', "shrink": .82}, 
                xticklabels=ca2_resn_one, yticklabels=ca1_resn_one, 
                vmin=0, vmax=40, center=10, linewidths=0.01, linecolor='white', cbar=True)
    ax.invert_yaxis()
    # ax.set_title(f'Aβ2/NKAW distance per residue {peptide}', fontdict={'fontsize':24, 'fontweight':'bold'})
    # ax.set_xlabel('NKAW Residue Index', fontdict={'fontsize':22, 'fontweight':'bold'})
    # ax.set_ylabel('Aβ2 Residue Index', fontdict={'fontsize':22, 'fontweight':'bold'})
    ax.figure.axes[-1].yaxis.label.set_size(18) #Change size of cbar label
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)  #Change size of cbar numbers    
        #Add colors to letters
    add_colors_to_letters(ax,fontsize,do_ylabel=True)
    for tick in ax.get_xticklabels():
            tick.set_fontweight('bold')
            tick.set_fontsize(fontsize)
    plt.tight_layout()
    plt.savefig(f'{plots_folder}1.Heatmaps/{filename}_ab2_peptide_distance.svg')
    plt.close()

### Contacts

In [ ]:
def contacts_protein_protein(u,chain1,chain2,filename=filename,peptide=peptide,data_folder=data_folder,plots_folder=plots_folder,fontsize=18,cmap_color='RdBu'):
    """ This function analyses the contacts between the two AB42 chains """
    # Calculate contacts between chains
    n_frames = len(u.trajectory)
    ca1 = chain1.select_atoms('name CA')
    ca2 = chain2.select_atoms('name CA')
    ca1_resn = ca1.resnames
    ca2_resn = ca2.resnames
    ca1_resn_one = [mda.lib.util.convert_aa_code(i) for i in ca1_resn]
    ca2_resn_one = [mda.lib.util.convert_aa_code(i) for i in ca2_resn]
    contact_counts = np.zeros((42, 42), dtype=int)
    for ts in u.trajectory:
        dist_matrix = mda.analysis.distances.distance_array(ca1.positions, ca2.positions)
        contacts = np.where(dist_matrix < 8.0)
        contact_counts[contacts] += 1

    # Save to df
    df_contact_counts = pd.DataFrame(contact_counts)
    df_contact_counts.to_csv(f'{data_folder}1.Heatmaps/df_{filename}_ab1_ab2_contacts.csv')

    # Generate contact heatmap with colorbar using seaborn
    fig, ax = plt.subplots(figsize=(13,13))
    sns.heatmap(contact_counts/14, cmap=cmap_color, ax=ax, cbar_kws={'label': ' Contacts (%)', 'shrink': .82},
                xticklabels=ca1_resn_one, yticklabels=ca2_resn_one,
                square=True, vmin=0, vmax=100, center=10, linewidths=0.01, linecolor='white', 
                cbar=True).set(title='AB1 vs AB2')
    ax.invert_yaxis()
    ax.set_title(f'Aβ1/Aβ2 contacts per residue ({peptide})', fontdict={'fontsize':24, 'fontweight':'bold'})
    ax.set_xlabel('Aβ2 Residue Index', fontdict={'fontsize':22, 'fontweight':'bold'})
    ax.set_ylabel('Aβ1 Residue Index', fontdict={'fontsize':22, 'fontweight':'bold'})
    ax.figure.axes[-1].yaxis.label.set_size(18) #Change size of cbar label
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)  #Change size of cbar numbers
    #Add colors to letters
    add_colors_to_letters(ax,fontsize,do_xlabel=True,do_ylabel=True)

    plt.tight_layout()
    plt.savefig(f'{plots_folder}1.Heatmaps/{filename}_ab1_ab2_contacts.svg')
    plt.close()

In [ ]:
def contacts_peptide_abeta1(u,chain1,chain2,chain3='',filename=filename,peptide=peptide,data_folder=data_folder,plots_folder=plots_folder,fontsize=18,cmap_color='RdBu'):
    """ This function analyses the contacts between the peptide and AB42 chain 1 """
    # Calculate contacts between chains
    n_frames = len(u.trajectory)
    ca1 = chain1.select_atoms('name CA')
    ca2 = chain3.select_atoms('name CA')
    ca1_resn = ca1.resnames
    ca2_resn = ca2.resnames
    ca1_resn_one = [mda.lib.util.convert_aa_code(i) for i in ca1_resn]
    ca2_resn_one = [mda.lib.util.convert_aa_code(i) for i in ca2_resn]
    contact_counts = np.zeros((len(ca1), len(ca2)), dtype=int)

    for ts in u.trajectory:
        dist_matrix = mda.analysis.distances.distance_array(ca1.positions, ca2.positions)
        contacts = np.where(dist_matrix < 8.0)
        contact_counts[contacts] += 1

    # Save to df
    df_contact_counts = pd.DataFrame(contact_counts)
    df_contact_counts.to_csv(f'{data_folder}1.Heatmaps/df_{filename}_ab1_peptide_contacts.csv')

    # Generate contact heatmap with colorbar using seaborn
    fig, ax = plt.subplots(figsize=(13,13))
    sns.heatmap(contact_counts/14, cmap=cmap_color, ax=ax, cbar_kws={'label': 'Contacts (%)', 'shrink': .82},
               xticklabels=ca2_resn_one, yticklabels=ca1_resn_one, 
                vmin=0, vmax=100, center=10, linewidths=0.01, linecolor='white', cbar=True)
    ax.invert_yaxis()
    # ax.set_title(f'Aβ1/NKAW contact per residue ({peptide})', fontdict={'fontsize':24, 'fontweight':'bold'})
    # ax.set_xlabel('NKAW Residue Index', fontdict={'fontsize':22, 'fontweight':'bold'})
    # ax.set_ylabel('Aβ1 Residue Index', fontdict={'fontsize':22, 'fontweight':'bold'})
    ax.figure.axes[-1].yaxis.label.set_size(18) #Change size of cbar label
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)  #Change size of cbar numbers
    #Add colors to letters
    add_colors_to_letters(ax,fontsize,do_ylabel=True)
    for tick in ax.get_xticklabels():
            tick.set_fontweight('bold')
            tick.set_fontsize(fontsize)
    plt.tight_layout()
    plt.savefig(f'{plots_folder}1.Heatmaps/{filename}_ab1_peptide_contacts.svg')
    plt.close()

In [ ]:
def contacts_peptide_abeta2(u,chain1,chain2,chain3='',filename=filename,peptide=peptide,data_folder=data_folder,plots_folder=plots_folder,fontsize=18,cmap_color='RdBu'):
    """ This function analyses the distance between the peptide and AB42 chain 2 """
    # Calculate contacts between chains
    n_frames = len(u.trajectory)
    ca1 = chain2.select_atoms('name CA')
    ca2 = chain3.select_atoms('name CA')
    ca1_resn = ca1.resnames
    ca2_resn = ca2.resnames
    ca1_resn_one = [mda.lib.util.convert_aa_code(i) for i in ca1_resn]
    ca2_resn_one = [mda.lib.util.convert_aa_code(i) for i in ca2_resn]

    contact_counts = np.zeros((len(ca1), len(ca2)), dtype=int)

    for ts in u.trajectory:
        dist_matrix = mda.analysis.distances.distance_array(ca1.positions, ca2.positions)
        contacts = np.where(dist_matrix < 8.0)
        contact_counts[contacts] += 1

    # Save to df
    df_contact_counts = pd.DataFrame(contact_counts)
    df_contact_counts.to_csv(f'{data_folder}1.Heatmaps/df_{filename}_ab2_peptide_contacts.csv')

    # Generate contact heatmap with colorbar using seaborn
    fig, ax = plt.subplots(figsize=(13,13))
    sns.heatmap(contact_counts/14, cmap=cmap_color, ax=ax, cbar_kws={'label': 'Contacts (%)', 'shrink': .82},
               xticklabels=ca2_resn_one, yticklabels=ca1_resn_one, 
                vmin=0, vmax=100, center=10, linewidths=0.01, linecolor='white', cbar=True).set(title='AB1 vs AB2')
    ax.invert_yaxis()
    # ax.set_title(f'Aβ2/NKAW contact per residue {peptide}', fontdict={'fontsize':24, 'fontweight':'bold'})
    # ax.set_xlabel('NKAW Residue Index', fontdict={'fontsize':22, 'fontweight':'bold'})
    # ax.set_ylabel('Aβ2 Residue Index', fontdict={'fontsize':22, 'fontweight':'bold'})
    ax.figure.axes[-1].yaxis.label.set_size(18) #Change size of cbar label
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=18)  #Change size of cbar numbers    
    #Add colors to letters
    add_colors_to_letters(ax,fontsize,do_ylabel=True)
    for tick in ax.get_xticklabels():
            tick.set_fontweight('bold')
            tick.set_fontsize(fontsize)
    plt.tight_layout()
    plt.savefig(f'{plots_folder}1.Heatmaps/{filename}_ab2_peptide_contacts.svg')
    plt.close()

## Secondary Structure

In [ ]:
def analyse_secondary_structure(folder=folder,pept_num=pept_num):
    """ This function analyses the secondary structure of the two AB42 chains and the peptide (if present) """
    colors = ['lightgreen','cornflowerblue','lightcoral','orchid','#b99ee8','thistle','khaki']
    with open(f"{folder}dssp.tml") as f:
        lines = f.readlines()[9:]
        residues = [int(line.split()[0]) for line in lines]
        frames = [int(line.split()[-2]) for line in lines]
        structures = [line.split()[-1] for line in lines]

    #Create pandas dataframe containing Residue, Frame and Structure information
    df = pd.DataFrame({"Residue": residues, "Frame": frames, "Structure": structures})

    #Using crosstab function to determine the frequency of ss per residue
    ss_data = pd.crosstab(df["Residue"], df["Structure"])

    #Computing ss percentages and storing them in another dataframe
    ss_percentage = ss_data.apply(lambda x: (x/len(u.trajectory))*100)

    ss_percentage = ss_percentage.rename(columns= {
        'B':'Isolated β Bridge',
        'C': 'Coil',
        'E': 'β Extended',
        'G': '3-10 Helix',
        'H': 'α Helix',
        'I': 'π Helix',
        'T': 'Turn'
    })

    ss_percentage = ss_percentage.reindex(columns=['Coil', 'Isolated β Bridge','β Extended','3-10 Helix','α Helix','π Helix','Turn'])
    ss_percentage.to_csv(f'{data_folder}2.SecStruct/{pept_num}.csv')

    # Save the residues in a variable
    abeta_residues = pd.read_csv('abeta_residues.csv',index_col=0)
    nka_residues = pd.read_csv('nka9_residues.csv',index_col=0)
    nkaw_residues = pd.read_csv('nkaw9_residues.csv',index_col=0)

    #Generate plot AB1
    barwidth = 1
    ax = ss_percentage.loc[:42].plot(kind='bar', stacked=True, figsize=(10, 6), color=colors,width=1.0, edgecolor='black')
    ax.set_xticklabels(list(abeta_residues['0']), rotation=0, ha='center') # Residue label
    # plt.ylabel("Percentage (%)", weight='bold', fontsize=12)
    # plt.xlabel('# Residue', weight='bold', fontsize=12)
    plt.ylim(0,100)
    # plt.title('Aβ2-NKA1: Aβ1', fontsize=16, weight='bold')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    add_colors_to_letters(ax,fontsize=12,do_xlabel=True)
    plt.savefig(f'{plots_folder}2.SecStruct/{pept_num}_ss_ab1.svg',transparent=True)
    plt.show()
    if pept_num == 0:
        #Generate plot AB2
        barwidth = 1
        ax = ss_percentage.loc[43:].plot(kind='bar', stacked=True, figsize=(10, 6), color=colors,width=1.0, edgecolor='black')
        ax.set_xticklabels(list(abeta_residues['0']), rotation=0, ha='center')
        # plt.ylabel("Percentage (%)", weight='bold', fontsize=12)
        # plt.xlabel('# Residue', weight='bold', fontsize=12)
        plt.ylim(0,100)
        # plt.title('Aβ2-NKA1: Aβ1', fontsize=16, weight='bold')
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        add_colors_to_letters(ax,fontsize=12,do_xlabel=True)
        plt.savefig(f'{plots_folder}2.SecStruct/{pept_num}_ss_ab2.svg',transparent=True)
        plt.close()
    elif pept_num == 1:
        #Generate plot AB2
        barwidth = 1
        ax = ss_percentage.loc[43:84].plot(kind='bar', stacked=True, figsize=(10, 6), color=colors,width=1.0, edgecolor='black')
        ax.set_xticklabels(list(abeta_residues['0']), rotation=0, ha='center')
        # plt.ylabel("Percentage (%)", weight='bold', fontsize=12)
        # plt.xlabel('# Residue', weight='bold', fontsize=12)
        plt.ylim(0,100)
        # plt.title('Aβ2-NKA1: Aβ1', fontsize=16, weight='bold')
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        add_colors_to_letters(ax,fontsize=12,do_xlabel=True)
        plt.savefig(f'{plots_folder}2.SecStruct/{pept_num}_ss_ab2.svg',transparent=True)
        plt.close()
        #Generate plot NKA
        barwidth = 1
        ax = ss_percentage.loc[85:].plot(kind='bar', stacked=True, figsize=(10, 6), color=colors,width=1.0, edgecolor='black')
        ax.set_xticklabels(list(nka_residues['0']), rotation=0, ha='center')
        # plt.ylabel("Percentage (%)", weight='bold', fontsize=12)
        # plt.xlabel('# Residue', weight='bold', fontsize=12)
        plt.ylim(0,100)
        # plt.title('Aβ2-NKA1: Aβ1', fontsize=16, weight='bold')
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        for tick in ax.get_xticklabels():
            tick.set_fontweight('bold')
            tick.set_fontsize(12)
        plt.savefig(f'{plots_folder}2.SecStruct/{pept_num}_ss_nka.svg',transparent=True)
        plt.close()
    elif pept_num == 3:
        #Generate plot
        barwidth = 1
        ax = ss_percentage.loc[52:].plot(kind='bar', stacked=True, figsize=(10, 6), color=colors,width=1.0, edgecolor='black')
        ax.set_xticklabels(list(abeta_residues['0']), rotation=0, ha='center')
        # plt.ylabel("Percentage (%)", weight='bold', fontsize=12)
        # plt.xlabel('# Residue', weight='bold', fontsize=12)
        plt.ylim(0,100)
        # plt.title('Aβ2', fontsize=16, weight='bold')
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        add_colors_to_letters(ax,fontsize=12,do_xlabel=True)
        plt.savefig(f'{plots_folder}2.SecStruct/{pept_num}_ss_ab2.svg',transparent=True)
        plt.close()

        #Generate plot
        barwidth = 1
        ax = ss_percentage.loc[43:51].plot(kind='bar', stacked=True, figsize=(10, 6), color=colors,width=1.0, edgecolor='black')
        ax.set_xticklabels(list(nkaw_residues['0']), rotation=0, ha='center')
        # plt.ylabel("Percentage (%)", weight='bold', fontsize=12)
        # plt.xlabel('# Residue', weight='bold', fontsize=12)
        plt.ylim(0,100)
        # plt.title('NKA', fontsize=16, weight='bold')
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        for tick in ax.get_xticklabels():
            tick.set_fontweight('bold')
            tick.set_fontsize(12)
        plt.savefig(f'{plots_folder}2.SecStruct/{pept_num}_ss_nkaw.svg',transparent=True)
        plt.close()
    
    else:
        #Generate plot
        barwidth = 1
        ax = ss_percentage.loc[52:93].plot(kind='bar', stacked=True, figsize=(10, 6), color=colors,width=1.0, edgecolor='black')
        ax.set_xticklabels(list(abeta_residues['0']), rotation=0, ha='center')
        # plt.ylabel("Percentage (%)", weight='bold', fontsize=12)
        # plt.xlabel('# Residue', weight='bold', fontsize=12)
        plt.ylim(0,100)
        # plt.title('Aβ2-NKA1: Aβ2', fontsize=16, weight='bold')
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        add_colors_to_letters(ax,fontsize=12,do_xlabel=True)
        plt.savefig(f'{plots_folder}2.SecStruct/{pept_num}_ss_ab2.svg',transparent=True)
        plt.show()

        if pept_num == 2:
            #Generate plot
            barwidth = 1
            nka1 = ss_percentage.loc[43:51]
            nka2 = ss_percentage.loc[94:]
            combined = pd.concat([nka1,nka2])
            
            ax = combined.plot(kind='bar', stacked=True, figsize=(10, 6), color=colors,width=1.0, edgecolor='black')
            ax.set_xticklabels(list(nka_residues['0'])+list(nka_residues['0']), rotation=0, ha='center')
            # plt.ylabel("Percentage (%)", weight='bold', fontsize=12)
            # plt.xlabel('# Residue', weight='bold', fontsize=12)
            plt.ylim(0,100)
            # plt.title('Aβ2-NKA1: NKA', fontsize=16, weight='bold')
            plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
            for tick in ax.get_xticklabels():
                tick.set_fontweight('bold')
                tick.set_fontsize(12)
            plt.savefig(f'{plots_folder}2.SecStruct/{pept_num}_ss_nka.svg',transparent=True)
            plt.show()
        elif pept_num == 4:
            #Generate plot
            barwidth = 1
            nka1 = ss_percentage.loc[43:51]
            nka2 = ss_percentage.loc[94:]
            combined = pd.concat([nka1,nka2])
            ax = combined.plot(kind='bar', stacked=True, figsize=(10, 6), color=colors,width=1.0, edgecolor='black')
            ax.set_xticklabels(list(nkaw_residues['0'])+list(nkaw_residues['0']), rotation=0, ha='center')
            # plt.ylabel("Percentage (%)", weight='bold', fontsize=12)
            # plt.xlabel('# Residue', weight='bold', fontsize=12)
            plt.ylim(0,100)
            # plt.title('Aβ2-NKA1: NKA', fontsize=16, weight='bold')
            plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
            for tick in ax.get_xticklabels():
                tick.set_fontweight('bold')
                tick.set_fontsize(12)
            plt.savefig(f'{plots_folder}2.SecStruct/{pept_num}_ss_nkaw.svg',transparent=True)
            plt.show()

## RMSD

In [ ]:
def analyse_rmsd(pept_num):
    """ This function analyses the RMSD of each AB42 chain and the peptide (if present) """
    folder = f'{folders[pept_num]}/amber/'
    parm = f'{folder}step3_input.pdb'
    if pept_num == 4: all_traj = [f'{folder}step5_1us.nc',f'{folder}step5_1us-2.nc',f'{folder}step5_1us-3.nc']
    elif pept_num == 3: all_traj = [f'{folder}try.nc',f'{folder}step5_1us-2.nc']
    elif pept_num == 0:  all_traj = [f'{folder}step5_1us.nc',f'{folder}step5_1us-2.nc']
    else: all_traj = [f'{folder}step5_1us.nc']
    filename = f'{filenames[pept_num]}'
    peptide = peptides[pept_num]

    # Run analysis
    u = mda.Universe(parm,all_traj)
    R = mda.analysis.rms.RMSD(u,select='name CA',groupselections=['segid PROA','segid PROB','segid PROC','segid PROD'])
    R.run()
    rmsd = R.rmsd.T # transpose makes it easier for plotting
    time = [i/10 for i in range(len(u.trajectory))]

    # Save to df
    df = pd.DataFrame()
    df['Time'] = time 
    df['PROA'] = rmsd[2]
    df['PROB'] = rmsd[3]
    if pept_num != 0:
        df['PROC'] = rmsd[4]
        if pept_num == 2 or pept_num == 4:
            df['PROD'] = rmsd[5]
    df.to_csv(f'{data_folder}4.RMSD/{pept_num}.csv')